In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import folium
import json
import os
from branca.colormap import linear
import branca.colormap

In [10]:
# US States GeoJSON file
us_states = os.path.join('us-states.json')
geo_json_data = json.load(open(us_states))

#Valid States 
valid_states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI',
       'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI',
       'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC',
       'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT',
       'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

In [4]:
# Donation Data
all_donations = pd.read_csv("2020 DONOR DATA.csv", header=None)

In [34]:
cleaned = all_donations[[0,9,14,21]].head(100000).sort_values(by=21).rename(columns=
                                                                             {0:"Committee ID" , 
                                                                              9:"State" , 
                                                                              14:"Amount"})

cleaned = cleaned[cleaned['State'].isin(valid_states)]
num_donations = cleaned.groupby("State").count().rename(columns={"Committee ID":"Count"})[["Count"]]
donation_dict = num_donations.to_dict().get('Count')

,Count
State,
AK,285
AL,663
AR,531
AZ,2284
CA,16690
CO,2287
CT,1344
DE,345
FL,4504


In [22]:
# Building Base Map
m = folium.Map([43, -100], zoom_start=4)

folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        'fillColor': 'white',
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(m)

In [54]:
colormap = linear.OrRd_03.scale(
    num_donations.Count.min(),
    num_donations.Count.max())

colormap

In [55]:
# Geographic Distribution of Donations 
m = folium.Map([43, -100], zoom_start=4)

folium.GeoJson(
    geo_json_data,
    name='donations',
    style_function=lambda feature: {
        'fillColor': colormap(donation_dict[feature['id']]),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }
).add_to(m)

folium.LayerControl().add_to(m)

m

In [49]:
avg_donation_amount = cleaned.groupby("State").mean()[["Amount"]]
avg_don_dict = avg_donation_amount.to_dict().get("Amount")
avg_donation_amount

,Amount
State,
AK,75.192982
AL,192.372549
AR,274.096045
AZ,74.958406
CA,110.861594
CO,156.972890
CT,96.670387
DE,76.800000
FL,117.823934


In [60]:
colormap = linear.BuGn_03.scale(
    avg_donation_amount.Amount.min(),
    300)

colormap

In [61]:
# Geographic Distribution of Donations 
m = folium.Map([43, -100], zoom_start=4)

folium.GeoJson(
    geo_json_data,
    name='avg donations',
    style_function=lambda feature: {
        'fillColor': colormap(avg_don_dict[feature['id']]),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }
).add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
# Use median to control for Nevada and Arkansas
# Breakdown for SuperPAC data